In [1]:
import matplotlib.pyplot as plt
import numpy as np
import casperfpga
import time
from lwa_f import snap2_fengine
from lwa_f import blocks
from lwautils import lwa_arx 
import pandas as pd
from cr_functions import *
import struct
import math
from scipy import special as sp
from scipy import optimize as oz
from scipy import stats as st


In [2]:

def packantennaroles(roles_array):
    #turn an array of 64 ones and zeros into two 32 bit integers, one for the first half and one for the second
    output1=0
    output2=0
    for i in range(32):
        output1+=roles_array[i]<<i
        output2+=roles_array[i+32]<<i

    return output1, output2

def setup_coincidencer(trigger_power_thresh,
                      veto_power_thresh,
                      trigger_window,
                      veto_window,
                      antenna_number_thresh,
                      veto_number_thresh,
                      delays,
                      core_roles_array,
                      veto_roles_array):
    # set parameters
    setvalue(casperbrd,"trigger_power_thresh","cr_registers.xlsx",trigger_power_thresh)
    setvalue(casperbrd,"veto_power_thresh","cr_registers.xlsx",veto_power_thresh)
    setvalue(casperbrd,'trigger_window','cr_registers.xlsx',trigger_window) 
    setvalue(casperbrd,'veto_window','cr_registers.xlsx',veto_window) 
    setvalue(casperbrd,'antenna_number_thresh','cr_registers.xlsx',antenna_number_thresh) 
    setvalue(casperbrd,'veto_number_thresh','cr_registers.xlsx',veto_number_thresh) 
    #set the antenna roles
    core_roles1,core_roles2=packantennaroles(core_roles_array)
    veto_roles1,veto_roles2=packantennaroles(veto_roles_array)
    setvalue(casperbrd,'trigger_antennas1','cr_registers.xlsx',core_roles1)
    setvalue(casperbrd,'trigger_antennas2','cr_registers.xlsx',core_roles2)
    setvalue(casperbrd,'veto_antennas1','cr_registers.xlsx',veto_roles1)
    setvalue(casperbrd,'veto_antennas2','cr_registers.xlsx',veto_roles2)

    #see notebook subarray_threshold_scans for a function to extract the delays from a text file of delays and antenna names
    delays=10*np.ones(64,dtype=int)
    delayblock=blocks.delay.Delay(casperbrd, 'delay', n_streams=64, logger=None)
    delayblock.initialize()
    for i in range(64):
        z=delays[i]
        delayblock.set_delay(i,z)
    return

def test_coincidencer(brd,trigger_power_thresh,
                      veto_power_thresh,
                      trigger_window,
                      veto_window,
                      antenna_number_thresh,
                      veto_number_thresh,
                      delays,
                      core_roles_array,
                      veto_roles_array):

    setup_coincidencer(trigger_power_thresh,
                          veto_power_thresh,
                          trigger_window,
                          veto_window,
                          antenna_number_thresh,
                          veto_number_thresh,
                          delays,
                          core_roles_array,
                          veto_roles_array)
    # reset trigger counter
    setvalue(brd,"trig_debug_reset","cr_registers.xlsx",1)
    time.sleep(0.003)
    setvalue(brd,"trig_debug_reset","cr_registers.xlsx",1)
    
    time.sleep(2.75)  #have to wait for it to update
    trigger_count=getvalue(brd,"count_triggers","cr_registers.xlsx")

    veto_coincidence,core_coincidence=struct.unpack('>2l',casperbrd.read("cosmic_ray_core_and_veto_rate",2*4,0))
    print("Core coincidence per 10b cycle: ", core_coincidence/262144)
    print("Veto coincidence per 10b cycle: ", veto_coincidence/262144)
    print("Triggers per 10b cycle (roughly)",trigger_count/262144)
    return

In [3]:
fpgfile='/home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-06-01_1510.fpg'
brdname='snap01'
brd=snap2_fengine.Snap2Fengine(brdname)
brd.program(fpgfile)
casperbrd = casperfpga.CasperFpga(brdname, transport=casperfpga.TapcpTransport)

2022-09-01 21:59:24,524 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-09-01 21:59:24,723 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1
2022-09-01 21:59:24,733 - lwa_f.snap2_fengine:snap01 - INFO - Loading firmware /home/ubuntu/kp/caltech-lwa/snap2_f_200msps_64i_4096c/outputs/snap2_f_200msps_64i_4096c_2022-06-01_1510.fpg to snap01
2022-09-01 21:59:46,367 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 0
2022-09-01 21:59:46,588 - lwa_f.blocks.block:snap01 - INFO - adc - Detected FMC ADC board on port 1


In [4]:
casperbrd.listdev()
brd.initialize(read_only=False)
brd.sync.arm_sync()
brd.sync.sw_sync()

2022-09-01 21:59:46,614 - lwa_f.snap2_fengine:snap01 - INFO - Initializing block (writable): fpga
2022-09-01 21:59:46,615 - lwa_f.snap2_fengine:snap01 - INFO - Initializing block (writable): adc
2022-09-01 21:59:46,618 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: Setting clock source to 1
2022-09-01 21:59:46,624 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1: Setting clock source to 1
2022-09-01 21:59:46,632 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 0: Setting clock source to 1
2022-09-01 21:59:46,639 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 board 1: Setting clock source to 1
2022-09-01 21:59:57,090 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0 clock rates: [490850954, 98238614, 98288630, 0, 0]
2022-09-01 21:59:57,091 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 0: MMCM locked
2022-09-01 21:59:58,102 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 board 1 clock rates: [490693575, 98208079, 98248660, 98280748, 0]
2022-09-

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-09-01 22:00:08,903 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane delays:
[[328, 320, 328, 312, 320, 320, 320, 312], [296, 288, 288, 272, 280, 280, 272, 280], [272, 272, 264, 264, 272, 272, 272, 280], [304, 312, 304, 296, 304, 304, 304, 304], [288, 280, 280, 288, 288, 280, 280, 288], [248, 256, 264, 256, 248, 248, 248, 248], [272, 248, 256, 264, 264, 272, 256, 256], [304, 296, 296, 296, 296, 304, 304, 296]]
2022-09-01 22:00:08,904 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 0 data lane slacks:
[[40, 40, 40, 40, 40, 40, 40, 32], [48, 40, 40, 40, 40, 40, 40, 40], [48, 40, 40, 40, 40, 40, 48, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40], [40, 40, 40, 40, 40, 40, 40, 40]]
2022-09-01 22:00:08,904 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|-----XXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,905 - lwa_f.blocks.block:snap01 - INF

2022-09-01 22:00:08,943 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,943 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,944 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,945 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,945 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,946 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:00:08,947 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 7, Lane 0:    XXXXXXXX

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


...

100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-09-01 22:01:10,575 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 0 because nowhere was good
2022-09-01 22:01:10,586 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 1 because nowhere was good
2022-09-01 22:01:10,599 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 2 because nowhere was good
2022-09-01 22:01:10,612 - lwa_f.blocks.block:snap01 - INFO - adc - Bitslipping board 0 chip 3 because nowhere was good
2022-09-01 22:01:11,585 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 Scanning data delays


100% (64 of 64) |########################| Elapsed Time: 0:00:07 Time:  0:00:07


2022-09-01 22:01:19,009 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane delays:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [280, 288, 288, 296, 288, 288, 288, 288], [232, 224, 224, 224, 232, 224, 232, 224], [272, 264, 264, 280, 264, 264, 264, 264], [304, 304, 304, 304, 304, 304, 304, 296]]
2022-09-01 22:01:19,010 - lwa_f.blocks.block:snap01 - INFO - adc - FMC 1 data lane slacks:
[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [40, 40, 48, 40, 40, 40, 40, 40], [40, 40, 32, 40, 40, 40, 40, 32], [32, 32, 32, 40, 32, 32, 32, 32], [40, 40, 40, 40, 40, 40, 40, 32]]
2022-09-01 22:01:19,011 - lwa_f.blocks.block:snap01 - WARNING - adc - Delay solutions have small slack
2022-09-01 22:01:19,012 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 0, Lane 0:    |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,013 - lwa_f.blocks.block:snap0

2022-09-01 22:01:19,050 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 1:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|---XXXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,051 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 2:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,052 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 3:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX----|----XXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,052 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 4:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,053 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 5:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,054 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 6:    XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX---|----XXXXXXXXXXXXXXXXXXXXXXXXXX
2022-09-01 22:01:19,054 - lwa_f.blocks.block:snap01 - INFO - adc - Chip 6, Lane 7:    XXXXXXXX

In [5]:
setup_data_source(casperbrd,"counter")

Data source set to counter


The data source is set to a counter. 
## First test
The threshold is set to 511, all antennas have both roles, the antenna number thresholds and the window thresholds are zero.  As expected, all antennas reach the threshold once per 1024 clock cycles, and so there is a trigger and a veto once every 1024 clock cycles, or 262144 times in every time interval between updating the counter register.

In [6]:
#define parameters
trigger_power_thresh=511**2
veto_power_thresh=511**2
trigger_window=100
veto_window=200
antenna_number_thresh=0
veto_number_thresh=0

#see notebook subarray_threshold_scans for a function to extract the delays from a text file of delays and antenna names
delays=10*np.ones(64,dtype=int)
core_roles_array=np.ones(64,dtype=int)
veto_roles_array=np.ones(64,dtype=int)

setup_coincidencer(trigger_power_thresh,
                      veto_power_thresh,
                      trigger_window,
                      veto_window,
                      antenna_number_thresh,
                      veto_number_thresh,
                      delays,
                      core_roles_array,
                      veto_roles_array)


time.sleep(2.75)  #have to wait for it to update
veto_coincidence,core_coincidence=struct.unpack('>2l',casperbrd.read("cosmic_ray_core_and_veto_rate",2*4,0))
core,veto = read_threshold_rates(casperbrd) #read each block

print("Core coincidence: ", core_coincidence)
print("Veto coincidence: ", veto_coincidence)
print(core[0],veto[1])
print(min(core),max(core))
print(min(veto),max(veto))
clockcycles_counted_for=(2**28)  #number of clock cycles that the trigger and veto rate counters counts for before updating
number_of_10bit_counter_cycles=clockcycles_counted_for/1024 #number of times the 10 bit counter starts over during the above count
print(clockcycles_counted_for)
print(number_of_10bit_counter_cycles)

Core coincidence:  262144
Veto coincidence:  262144
262144.0 262144.0
262144.0 262144.0
262144.0 262144.0
268435456
262144.0


In [7]:
#repeat above, in order to test out the test function
trigger_power_thresh=511**2
veto_power_thresh=511**2
trigger_window=100
veto_window=200
antenna_number_thresh=0
veto_number_thresh=0


test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)

Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


# Main Tests

Trigger and veto power thresholds remain 511, and the core and veto number thresholds remain 2 (at least three antennas required).  The core antennas are signals 0, 1, and 2, and the veto antennas are signals 3, 4, 5.  The next sections go through five scenarios with variants.






In [8]:
trigger_power_thresh=511**2
veto_power_thresh=511**2
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:3]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[3:6]=1
antenna_number_thresh=2
veto_number_thresh=2

## 1. Veto after Core

### a) Veto long after

In [9]:
delays=np.ones(64,dtype=int)
delays[0:6]=[1,5,10,450,500,511] #core antennas are first three, veto antennas are second three


print("1) Expect Trigger is vetoed")
trigger_windows=[11,50,100,200,300,400,500]
veto_windows=[511,450,400,300,200,100,62]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')


print("2) Expect Veto misses trigger")
trigger_windows=[11,20,100,11]
veto_windows=[62,100,300,350]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')

print("3) Expect No trigger or veto")
trigger_windows=[10,9,2]
veto_windows=[61,60,49]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')


1) Expect Trigger is vetoed
trigger window:  11 veto window:  511
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  50 veto window:  450
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  100 veto window:  400
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  200 veto window:  300
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  300 veto window:  200
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  400 veto window:  100
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  500 veto window:  62
Core coincidence per 10b cycle

### Summarize problems
There is never a trigger that makes it to the trigger counter, even when no veto is detected.  In the scenarios where a veto should cancel a core trigger, the veto coincidence does not occur for the largest windows tested: 400, 450, 511.  For scenarios where the veto should miss the trigger, the veto similarly does not occur for the largest window value, 350.  For scenarios where no trigger or veto coincidence is expected, there is always a trigger and a veto.

### Troubleshooting
What is window length where the veto stops working?  Tests below show that this is 343. Next, I make a new section (at the end of the notebook) running the same tests as I simulated in Simulink.

In [10]:
trigger_windows=[300,310,330,340,350]
veto_windows=[300,310,330,340,350]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')

trigger window:  300 veto window:  300
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  310 veto window:  310
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  330 veto window:  330
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  340 veto window:  340
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  350 veto window:  350
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0




In [11]:
trigger_windows=[340,341,342,343,345,347,350]
veto_windows=[340,341,342,343,345,347,350]
for i in range(len(trigger_windows)):
    trigger_window=trigger_windows[i]
    veto_window=veto_window=veto_windows[i]
    print("trigger window: ", trigger_window, "veto window: ", veto_window)
    test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)
    print('')
print('')

trigger window:  340 veto window:  340
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  341 veto window:  341
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  342 veto window:  342
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  343 veto window:  343
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  345 veto window:  345
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  347 veto window:  347
Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0

trigger window:  350 veto window:  350
Core coincidence per 10b cycle:  1.0
Veto coincidence p

### b) Veto Soon after

### c) Veto widely arranged

## 2. Veto before core

### a) Veto long before


### b) Veto soon before

### c) Veto widely arranged

## 3. Veto antennas on either side of core

## 4. Veto antennas on either side and among core

## 5. Veto antennas among core

## Match the Simulink tests
Repeating all the tests in the August 23 Evernote note, but with the real system instead of Simulink.

A couple modifications to make an analogous test work with the 10 bit counter as signal:

--Voltage thresholds are always 511.
--Delays are 100,110,115,117,120 instead of 1000,1010,1015,1017,1020



In [12]:
delays=np.ones(64,dtype=int)
delays[0:5]=[100,110,115,117,120] #core antennas are first three, veto antennas are second three
trigger_power_thresh=511**2
veto_power_thresh=511**2

### Test A

In [13]:

core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=0
veto_number_thresh=0
trigger_window=20
veto_window=7
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  There was a core and veto coincidence, matching the simulation, however there is no final trigger.

### Test B

In [14]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=0
veto_number_thresh=0
trigger_window=7
veto_window=20
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, as expected and matching simulation.

### Test C

In [15]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=20
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, as expected and matching simulation.

### Test D

In [16]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[0:2]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[2:5]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=12
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, as expected and matching simulation.

### Test E

In [21]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[2:5]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[0:2]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=12
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  1.0
Triggers per 10b cycle (roughly) 0.0


Result:  Core and Veto both had a coincidence, with no final trigger, but the simulation did have a final trigger.

Next I confirm that the veto stops working beyond 341, even for parameters that are otherwise the same as E.

In [22]:
core_roles_array=np.zeros(64,dtype=int)
core_roles_array[2:5]=1
veto_roles_array=np.zeros(64,dtype=int)
veto_roles_array[0:2]=1
antenna_number_thresh=1
veto_number_thresh=1
trigger_window=11
veto_window=350
test_coincidencer(casperbrd,trigger_power_thresh,veto_power_thresh,trigger_window,veto_window,antenna_number_thresh,veto_number_thresh,delays,core_roles_array,veto_roles_array)


Core coincidence per 10b cycle:  1.0
Veto coincidence per 10b cycle:  0.0
Triggers per 10b cycle (roughly) 0.0


## More NOtes
Antenna number thresholds are greater than. If 3 is set to the register, then four antennas are the minimum for a trigger.

In [23]:
(11<<16)+350

721246